In [6]:
!pip install spacy PyMuPDF reportlab


In [11]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
!pip install pymupdf


In [13]:
!pip install scikit-learn


In [14]:
# -----------------------------
# STEP 0: Install Required Libraries
# -----------------------------
!pip install spacy pymupdf scikit-learn
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 57.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('e

In [15]:
import fitz  # PyMuPDF
import spacy
import re
from sklearn.metrics import precision_score, recall_score, f1_score
from google.colab import files
import pandas as pd


In [16]:
# STEP 2: Load SLM (start with small)
# -----------------------------
slm_model = "en_core_web_sm"
nlp = spacy.load(slm_model)

In [17]:
# STEP 3: Define regex patterns
# -----------------------------
patterns = {
    "PHONE": r"\b\d{10}\b",
    "AADHAR": r"\b\d{4}\s\d{4}\s\d{4}\b",
    "EMAIL": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
}

In [18]:
# STEP 4: Detect sensitive info
# -----------------------------
def detect_sensitive(text):
    sensitive = []

    # NER from SLM
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG", "GPE", "DATE"]:
            sensitive.append((ent.text, ent.start_char, ent.end_char))

    # Regex patterns
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            sensitive.append((match.group(), match.start(), match.end()))

    return sensitive

In [19]:
# STEP 5: Redact PDF and create a table of sensitive info
# -----------------------------
def redact_pdf(input_pdf, output_pdf):
    doc = fitz.open(input_pdf)
    table_data = []

    for page_num, page in enumerate(doc, start=1):
        text = page.get_text()
        sensitive_instances = detect_sensitive(text)

        for inst in sensitive_instances:
            words = page.search_for(inst[0])
            for rect in words:
                page.add_redact_annot(rect, fill=(0,0,0))
            # Add to table
            table_data.append([page_num, inst[0]])

        page.apply_redactions()
        # Add margin label
        page.insert_text((50, 50), "Sensitive information", fontsize=12, color=(1,0,0))

    doc.save(output_pdf)
    print(f"Redacted PDF saved as {output_pdf}")

    # Create a table of detected sensitive info
    df = pd.DataFrame(table_data, columns=["Page Number", "Sensitive Info"])
    display(df)
    return df

In [20]:
# STEP 6: Upload PDF in Colab
# -----------------------------
uploaded = files.upload()
for file_name in uploaded.keys():
    input_pdf = file_name
    output_pdf = "redacted_" + file_name

    # Redact PDF
    df = redact_pdf(input_pdf, output_pdf)

    # Download redacted PDF
    files.download(output_pdf)

Saving UNDERTAKING for Provisional Certificate[1].pdf to UNDERTAKING for Provisional Certificate[1] (1).pdf
Redacted PDF saved as redacted_UNDERTAKING for Provisional Certificate[1] (1).pdf


,Page Number,Sensitive Info
0,1,University of Petroleum and Energy Studies
1,1,Dehradun
2,1,Controller of Examination
3,1,about \n \n \nyears
4,1,SAP ID
5,1,the University of Petroleum and Energy Studies
6,1,Dehradun
7,1,SR
8,1,Student’s Name \n \n
9,1,Student’s Signature \n \n


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>